## import libraries

In [5]:
from docplex.mp.model import Model
import pandas as pd
import pickle

ModuleNotFoundError: No module named 'docplex'

## load data

In [2]:
aircraft_data = pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Aircraft.csv")
airports_data = pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Airports.csv", encoding='ISO-8859-1')
demands_file =  pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Demands.csv")
distance_time_file =  pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Distance and Flying Time.csv")

## preprocess

In [3]:
aircraft_set = aircraft_data['AircraftID'].unique()
demands_set = demands_file['DemandID'].unique()  # Corrected to 'demands_file'
airports_set = airports_data['Airport Name'].unique()  # Ensure 'Airport Name' is the correct column name
time_periods = pd.to_datetime(demands_file['ScheduledDepDatetime']).dt.date.unique()  # Corrected column name

## parameters

In [4]:
fixed_cost_per_leg = 10000000
# Operating cost for each flight leg (assumed to be in Distance and Flying Time data)
operating_costs = {row['DepAirport'] + row['ArrAirport']: row['FlyingTime']
                   for _, row in distance_time_file.iterrows()}
#
# Revenue from fulfilling demand (simplification: proportional to PaxCount)
revenue_per_demand = {row['DemandID']: row['PaxCount'] * 100  # Arbitrary revenue per passenger
                      for _, row in demands_file.iterrows()}

## Model initialization

In [5]:
model = LpProblem("Aircraft_Route_Optimization", LpMaximize)

## Decision Variables

In [6]:
L = list(distance_time_file[['DepAirport', 'ArrAirport']].drop_duplicates().itertuples(index=False, name=None))


In [7]:
x_vars = LpVariable.dicts("x", [(a, l, t) for a in aircraft_set for l in L for t in time_periods],cat=LpBinary)
y_vars = LpVariable.dicts("y", [(d, t) for d in demands_set for t in time_periods], cat=LpBinary)
z_vars = LpVariable.dicts("z", [(a, l) for a in aircraft_set for l in L], cat=LpBinary)

## Objective Function

In [19]:
# Revenue from fulfilling demands
revenue = lpSum(revenue_per_demand[d] * y_vars[d, t] for d in demands_set for t in time_periods)

# Operating costs for flying legs
cost = lpSum(operating_costs[l] * x_vars[a, l, t] for a in aircraft_set for l in L for t in time_periods)

# Objective function: Maximize profit (Revenue - Cost)
model += revenue - cost


KeyError: ('KGRB', 'KPWK')